In [1]:
from itertools import permutations, product

import tqdm

import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import os
from os import path

import matplotlib.pyplot as plt

from warnings import simplefilter
from collections import OrderedDict

if (os.path.abspath('').split('/')[-1] == 'project'):
    %cd utils
elif (os.path.abspath('').split('/')[-1] == 'train_and_vis'):
    %cd ../utils

import query_utils
import model_utils
import validation_utils

print(os.path.abspath(''))

if (os.path.abspath('').split('/')[-1] == 'utils'):
    %cd ..


simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action='ignore', category=FutureWarning)

/Users/benjaminli/Documents/coding/scires/project/utils
/Users/benjaminli/Documents/coding/scires/project/utils
/Users/benjaminli/Documents/coding/scires/project


In [2]:
nard_seq = pd.read_csv("data/nardus_sequences.csv")
# nard_seq.drop(0, axis=1, inplace=True)

In [3]:
print(nard_seq)

               ID                                       DNA Sequence  \
0     NC_025403.1  accagagggaaaatataacaatgtcgttttatagcgatgtaaataa...   
1     NC_025404.1  accagagggaaaattaagaaaggtcgttccaagacgacttaaaaga...   
2     NC_028246.1  acggagaaaaacaaaaaaactatagtgattagataaataaggaaaa...   
3     NC_002077.1  ttgcccactccctctctgcgcgctcgctcgctcggtggggcctgcg...   
4     NC_006152.1  ctctcccccctgtcgcgttcgctcgctcgctggctcgtttgggggg...   
...           ...                                                ...   
1843  NC_007661.1  gttaaaactctcacacttatggtggaactggatctgcaaaaatggg...   
1844  NC_007662.1  gttaaaaatctggttgtatctactcttgaatgaacgtgcataaagc...   
1845  NC_007663.1  gttaaaatctggaaccaatatggaagggatttatgcacgtgctttt...   
1846  NC_007664.1  gttaaaaaagagtgcagatgtcgagaatagtcttactaacaccagg...   
1847  NC_007665.1  gttaaaaacctcaagatgcatgccgctattacttcaatgaaacgtg...   

      isZoonotic  
0              0  
1              0  
2              0  
3              1  
4              1  
...          ...  
18

In [20]:
df = pd.read_csv('data/dataset1/human_infecting_virus', delimiter='\t', header=None)
df[['ID', 'DNA Sequence']] = df[0].str.split(expand=True)
df = df.drop(0, axis=1)
df['isZoonotic'] = 1

df2 = pd.read_csv('data/dataset1/Other_viruses', delimiter='\t', header=None)
df2[['ID', 'DNA Sequence']] = df2[0].str.split(expand=True)
df2 = df2.drop(0, axis=1)
df2['isZoonotic'] = 0

dfother = pd.DataFrame()
it = len(df)

mergedDf = pd.concat([df, df2, nard_seq], axis=0, ignore_index=True)
mergedDf = mergedDf.drop_duplicates(subset=['DNA Sequence'])
mergedDf.reset_index(drop=True, inplace=True)

In [21]:
# 1848
print(df.columns)
print(nard_seq.columns)

Index(['ID', 'DNA Sequence', 'isZoonotic'], dtype='object')
Index(['ID', 'DNA Sequence', 'isZoonotic'], dtype='object')


In [22]:
print(mergedDf)

                ID                                       DNA Sequence  \
0            37121  cttttaacaatcatagttttataaaagggtgtaaccgaagcggttt...   
1           129724  atactttacaattatcttgtaaaaagtagggtgtaaccgaaaaggg...   
2            59303  atggcggacgtgtgacatcaccgttcgctctttctaggatcctttg...   
3           120686  ttgttctacttcttactcattattataaattataatgtttgtataa...   
4            99565  gtgaatgatgatggcgtcgaaagacgtcgttgcagctactgctgct...   
...            ...                                                ...   
10622  NC_007661.1  gttaaaactctcacacttatggtggaactggatctgcaaaaatggg...   
10623  NC_007662.1  gttaaaaatctggttgtatctactcttgaatgaacgtgcataaagc...   
10624  NC_007663.1  gttaaaatctggaaccaatatggaagggatttatgcacgtgctttt...   
10625  NC_007664.1  gttaaaaaagagtgcagatgtcgagaatagtcttactaacaccagg...   
10626  NC_007665.1  gttaaaaacctcaagatgcatgccgctattacttcaatgaaacgtg...   

       isZoonotic  
0               1  
1               1  
2               1  
3               1  
4               1  
...